# Oct 27

🔍 What's Missing From Our Original Plan (OSIC-Only)
Looking back at our discussion, here's what you haven't done yet that we agreed was feasible:
* .Proper data splitting - You only did simple train/val split. We discussed 5-fold cross-validation OR holding out a proper test set (20%) that you evaluate ONCE at the end. Right now you have no true held-out evaluation.\n
* .Clinical baseline comparison - You need to train a clinical-only model (Age, Sex, Smoking, baseline FVC) and show that adding CT scans improves predictions. This proves CT adds value beyond simple demographics.\n
* .Comprehensive evaluation metrics - You only reported R², MAE, LLL. Missing: classification metrics (accuracy, sensitivity, specificity for fast/moderate/slow progressors), confusion matrix, calibration plots, and prediction scatter plots.\n
* .Interpretability/Visualization - No attention maps showing which lung regions drive predictions, no feature importance analysis, no training curves saved as figures.\n
* .Error analysis - Which patients did the model fail on? Are there patterns in failures? This builds trust and shows limitations honestly.\n
Do these 5 things with your current code and OSIC dataset, and you'll have a complete, publishable story showing that baseline CT scans can predict fibrosis progression! 🎯



In [1]:
# ============================================================================
#
#  OSIC Paper Review Script - Reproducing the v1.txt Result (R²=0.46)
#  - FIX: Corrected IndexError on targets[p][2] -> targets[p][1]
#
# ============================================================================

import os
import cv2
import pydicom
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import random
from tqdm import tqdm 
from sklearn.model_selection import train_test_split # Use single split
from sklearn.metrics import (r2_score, mean_absolute_error, mean_squared_error)
import xgboost as xgb
from scipy import stats
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
from torch.cuda.amp import GradScaler, autocast
from torch.optim.lr_scheduler import ReduceLROnPlateau 
from pathlib import Path
import albumentations as albu
from albumentations.pytorch import ToTensorV2
import warnings

warnings.filterwarnings('ignore')

# ============================================================================
# 1. CONFIGURATION (Matching v1.txt)
# ============================================================================

class Config:
    SEED = 42
    DATA_DIR = Path("/kaggle/input/osic-pulmonary-fibrosis-progression")
    TRAIN_DIR = DATA_DIR / "train" 
    OUTPUT_DIR = Path("./")
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # --- Split from v1.txt ---
    TEST_SIZE = 0.15 
    
    IMG_SIZE = 384
    BATCH_SIZE = 8
    NUM_WORKERS = 2
    
    EPOCHS = 50 # v1.txt ran for 50
    LR = 1e-4 # v1.txt LR
    WEIGHT_DECAY = 1e-4
    EARLY_STOP_PATIENCE = 10 # v1.txt used 10
    SCHEDULER_PATIENCE = 5 # v1.txt used 5
    
    # Dropout from v1.txt
    FUSION_DROPOUT = 0.2 

config = Config()

def seed_everything(seed=Config.SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

seed_everything()

print(f"🚀 REPRODUCING v1.txt (R²={0.46}, MAE={3.27})")
print("=" * 60)
print(f"📱 Device: {config.DEVICE}")
print(f"📐 Image Size: {config.IMG_SIZE}x{config.IMG_SIZE}")


# ============================================================================
# 2. DATA PREPARATION (from v1.txt)
# ============================================================================

def get_optimized_tab_features(df_row):
    vector = []
    age = df_row['Age']
    vector.extend([(age - 50) / 30, age / 100])
    vector.append(1.0 if df_row['Sex'] == 'Male' else 0.0)
    
    smoking_status = df_row['SmokingStatus']
    if smoking_status == 'Never smoked': vector.extend([1, 0, 0])
    elif smoking_status == 'Ex-smoker': vector.extend([0, 1, 0])
    elif smoking_status == 'Currently smokes': vector.extend([0, 0, 1])
    else: vector.extend([0, 0, 0])
    
    if 'FVC' in df_row:
        fvc = df_row['FVC']
        vector.extend([fvc / 3000, (fvc - 2500) / 1000])
    
    if 'FVC' in df_row and 'Age' in df_row:
        fvc, age, sex = df_row['FVC'], df_row['Age'], df_row['Sex']
        if sex == 'Male': pp_fvc = fvc / (27.63 - 0.112 * age) if age > 0 else 0.8
        else: pp_fvc = fvc / (21.78 - 0.101 * age) if age > 0 else 0.8
        vector.append(min(pp_fvc, 2.0))
    
    expected_dim = 9 
    while len(vector) < expected_dim:
        vector.append(0.0) 

    return np.array(vector[:expected_dim]) 

def get_progression_class(slope):
    """Categorizes slope for stratification."""
    if slope > -5: return 0  # Slow
    elif slope > -15: return 1  # Moderate
    else: return 2  # Fast

def prepare_data():
    print("Loading data and calculating targets...")
    df = pd.read_csv(config.DATA_DIR / 'train.csv')
    baseline_df = df.loc[df.groupby('Patient')['Weeks'].idxmin()]
    
    targets = {}
    tabular_dim = 0 
    
    for patient in tqdm(df['Patient'].unique()):
        sub = df[df['Patient'] == patient].copy().sort_values('Weeks')
        fvc, weeks = sub['FVC'].values, sub['Weeks'].values
        
        if len(weeks) >= 2:
            try:
                slopes = []
                for i in range(len(weeks)):
                    for j in range(i + 1, len(weeks)):
                        if weeks[j] != weeks[i]:
                            slopes.append((fvc[j] - fvc[i]) / (weeks[j] - weeks[i]))
                slope = np.median(slopes) if slopes else 0.0
            except Exception:
                slope = 0.0
        else:
            slope = 0.0
            
        tabular_features = get_optimized_tab_features(sub.iloc[0])
        if tabular_dim == 0:
            tabular_dim = len(tabular_features)
            
        # Store as 2-element tuple: (slope, features)
        targets[patient] = (slope, tabular_features)

    # Create bins for stratification (4 bins)
    slopes = [targets[p][0] for p in baseline_df['Patient'].values if p in targets]
    # Ensure baseline_df only contains patients we have targets for
    baseline_df = baseline_df[baseline_df['Patient'].isin(targets.keys())]
    baseline_df['decay_bins'] = pd.cut(slopes, bins=4, labels=False)
    
    print(f"Processed {len(targets)} patients.")
    print(f"Detected Tabular Dimension: {tabular_dim}")
    
    return df, targets, baseline_df, tabular_dim

# ============================================================================
# 3. DATASET & AUGMENTATIONS (from v1.txt)
# ============================================================================

class OptimizedAugmentation:
    def __init__(self, augment=True):
        if augment:
            self.transform = albu.Compose([
                albu.Rotate(limit=10, p=0.5),
                albu.HorizontalFlip(p=0.4),
                albu.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=10, p=0.6),
                albu.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.4),
                albu.GaussNoise(var_limit=(5.0, 20.0), p=0.3),
                albu.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ToTensorV2()
            ])
        else:
            self.transform = albu.Compose([
                albu.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ToTensorV2()
            ])
    
    def __call__(self, image):
        return self.transform(image=image)['image']

class OptimizedOSICDataset(Dataset):
    def __init__(self, patients, targets_dict, data_dir, split='train'):
        self.patients = [p for p in patients if p not in ['ID00011637202177653955184', 'ID00052637202186188008618']]
        self.targets_dict = targets_dict
        self.data_dir = Path(data_dir)
        self.split = split
        self.augmentor = OptimizedAugmentation(augment=(split=='train'))
        
        self.patient_images = {}
        for patient in self.patients:
            patient_dir = self.data_dir / patient
            if patient_dir.exists():
                self.patient_images[patient] = [f for f in patient_dir.iterdir() if f.suffix.lower() == '.dcm']
        
        self.valid_patients = [p for p in self.patients if p in self.patient_images and self.patient_images[p]]
        
        print(f"Dataset split '{split}': {len(self.valid_patients)} valid patients with images.")
    
    def __len__(self):
        return len(self.valid_patients) * 8 if self.split == 'train' else len(self.valid_patients)
    
    def __getitem__(self, idx):
        patient_idx = idx % len(self.valid_patients) if self.split == 'train' else idx
        patient = self.valid_patients[patient_idx]
        
        available_images = self.patient_images[patient]
        selected_image = random.choice(available_images)
        
        img = self.load_dicom(selected_image)
        img_tensor = self.augmentor(img)
        
        slope, tab_features = self.targets_dict[patient]
        
        return {
            "image": img_tensor,
            "tabular": torch.tensor(tab_features, dtype=torch.float32),
            "slope_target": torch.tensor(slope, dtype=torch.float32),
        }
    
    def load_dicom(self, path):
        try:
            dcm = pydicom.dcmread(str(path))
            img = dcm.pixel_array.astype(np.float32)
            
            img = cv2.resize(img, (config.IMG_SIZE, config.IMG_SIZE))
            
            img_min, img_max = img.min(), img.max()
            if img_max > img_min:
                img = (img - img_min) / (img_max - img_min) * 255
            else:
                img = np.zeros_like(img)
            
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
            img = clahe.apply(img.astype(np.uint8))
            img = np.stack([img, img, img], axis=2).astype(np.uint8)
            return img
        except Exception as e:
            print(f"Error loading {path}: {e}")
            return np.zeros((config.IMG_SIZE, config.IMG_SIZE, 3), dtype=np.uint8)

# ============================================================================
# 4. MODEL (from v1.txt)
# ============================================================================

class OptimizedDenseNetModel(nn.Module):
    def __init__(self, tabular_dim):
        super(OptimizedDenseNetModel, self).__init__()
        densenet = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
        self.features = densenet.features
        
        # --- Partial freezing as in v1.txt ---
        print("Applying partial freezing (like v1.txt): only first 100 layers frozen.")
        for i, param in enumerate(self.features.parameters()):
            param.requires_grad = i > 100 
        
        self.global_pool = nn.AdaptiveAvgPool2d(1)
        
        self.tabular_processor = nn.Sequential(
            nn.Linear(tabular_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2), 
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
        )
        
        self.fusion_layer = nn.Sequential(
            nn.Linear(1024 + 256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            # --- Dropout from v1.txt ---
            nn.Dropout(config.FUSION_DROPOUT), 
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
        )
        
        self.mean_head = nn.Sequential(
            nn.Linear(256, 128), nn.ReLU(),
            nn.Linear(128, 64), nn.ReLU(),
            nn.Linear(64, 1)
        )
        self.log_var_head = nn.Sequential(
            nn.Linear(256, 32), nn.ReLU(),
            nn.Linear(32, 1),
            nn.Tanh()
        )
        
        self._initialize_weights()
    
    def _initialize_weights(self):
        for m in [self.tabular_processor, self.fusion_layer, self.mean_head, self.log_var_head]:
            for layer in m:
                if isinstance(layer, nn.Linear):
                    nn.init.normal_(layer.weight, mean=0.0, std=0.01)
                    if layer.bias is not None:
                        nn.init.constant_(layer.bias, 0.0)
    
    def forward(self, images, tabular):
        img_features = self.features(images) 
        img_features = self.global_pool(img_features).view(images.size(0), -1)
        tab_features = self.tabular_processor(tabular)
        combined_features = torch.cat([img_features, tab_features], dim=1)
        fused_features = self.fusion_layer(combined_features)
        
        mean_pred = self.mean_head(fused_features).squeeze(-1)
        log_var = self.log_var_head(fused_features).squeeze(-1)
        
        return mean_pred, log_var

# ============================================================================
# 5. LOSS & METRICS (from v1.txt)
# ============================================================================

class RegressionLoss(nn.Module):
    """Laplace NLL loss from v1.txt."""
    def __init__(self):
        super().__init__()

    def forward(self, mean_pred, log_var, targets):
        # --- This is the v1.txt loss logic ---
        var = torch.exp(log_var)
        mse_loss_term = 0.5 * (mean_pred - targets) ** 2 / var
        unc_loss_term = 0.5 * log_var
        
        loss = mse_loss_term + unc_loss_term
        
        loss = torch.where(torch.isinf(loss) | torch.isnan(loss), torch.zeros_like(loss), loss)
        return loss.mean()


def calculate_laplace_lll(actual, predicted, sigma):
    sigma = np.maximum(sigma, 1e-6)
    delta = np.abs(actual - predicted)
    lll = -np.sqrt(2) * delta / sigma - np.log(sigma * np.sqrt(2))
    finite_lll = lll[np.isfinite(lll)]
    return np.mean(finite_lll) if len(finite_lll) > 0 else -np.inf


def calculate_all_metrics(results):
    metrics = {}
    slope_true = results['slope_targets']
    slope_pred = results['slope_preds']

    finite_mask = np.isfinite(slope_pred) & np.isfinite(slope_true)
    if np.sum(finite_mask) < 2:
         return {'R2': 0.0, 'MAE': np.inf, 'MSE': np.inf, 'RMSE': np.inf, 'LaplaceLL': -np.inf}
        
    slope_true_f = slope_true[finite_mask]
    slope_pred_f = slope_pred[finite_mask]

    metrics['R2'] = r2_score(slope_true_f, slope_pred_f)
    metrics['MAE'] = mean_absolute_error(slope_true_f, slope_pred_f)
    metrics['MSE'] = mean_squared_error(slope_true_f, slope_pred_f)
    metrics['RMSE'] = np.sqrt(metrics['MSE'])
    
    log_var_preds_f = results['log_var_preds'][finite_mask]
    sigma_pred_f = np.exp(log_var_preds_f / 2)
    metrics['LaplaceLL'] = calculate_laplace_lll(slope_true_f, slope_pred_f, sigma_pred_f)
        
    return metrics

# ============================================================================
# 6. TRAINING & VALIDATION LOOPS
# ============================================================================

def train_one_epoch(model, loader, optimizer, criterion, scaler):
    model.train()
    total_loss = 0
    batches_processed = 0
    pbar = tqdm(loader, desc="Training", leave=False)

    for batch in pbar:
        optimizer.zero_grad(set_to_none=True)

        with autocast(enabled=True):
            mean_pred, log_var = model(
                batch['image'].to(config.DEVICE),
                batch['tabular'].to(config.DEVICE)
            )
            loss = criterion(
                mean_pred, log_var,
                batch['slope_target'].to(config.DEVICE)
            )

        if torch.isnan(loss) or torch.isinf(loss):
            print(f"Warning: NaN or Inf loss detected in training batch. Skipping batch.")
            continue

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()
        batches_processed += 1
        pbar.set_postfix(loss=loss.item())

    return total_loss / batches_processed if batches_processed > 0 else 0.0


def validate_one_epoch(model, loader):
    model.eval()
    results = {'slope_preds': [], 'log_var_preds': [], 'slope_targets': []}

    with torch.no_grad():
        for batch in tqdm(loader, desc="Validating", leave=False):
            with autocast(enabled=True):
                mean_pred, log_var = model(
                    batch['image'].to(config.DEVICE),
                    batch['tabular'].to(config.DEVICE)
                )

            if mean_pred.ndim == 0:
                results['slope_preds'].append(mean_pred.cpu().item())
                results['log_var_preds'].append(log_var.cpu().item())
            else:
                results['slope_preds'].extend(mean_pred.cpu().tolist())
                results['log_var_preds'].extend(log_var.cpu().tolist())

            results['slope_targets'].append(batch['slope_target'].numpy())

    results['slope_preds'] = np.array(results['slope_preds'])
    results['log_var_preds'] = np.array(results['log_var_preds'])
    results['slope_targets'] = np.concatenate(results['slope_targets'])

    metrics = calculate_all_metrics(results)
    return metrics




🚀 REPRODUCING v1.txt (R²=0.46, MAE=3.27)
📱 Device: cuda
📐 Image Size: 384x384


In [2]:
# ============================================================================
# 7. MAIN EXECUTION (Single Split)
# ============================================================================

def run():
    df, targets, baseline_df, tabular_dim = prepare_data()
    
    patients = baseline_df['Patient'].values
    bins = baseline_df['decay_bins'].values
    
    # --- Create the v1.txt split ---
    train_p, val_p = train_test_split(
        patients, 
        test_size=config.TEST_SIZE, 
        random_state=config.SEED, 
        stratify=bins
    )
    
    print(f"\n--- Single Split (v1.txt replication) ---")
    print(f"Train: {len(train_p)} patients, Val: {len(val_p)} patients")
    
    # --- 1. Clinical Baseline (XGBoost) ---
    print("\n--- Training Clinical Baseline (XGBoost) ---")
    
    # --- *** FIX HERE *** ---
    # `targets[p][1]` is tabular_features
    # `targets[p][0]` is slope
    X_train = np.array([targets[p][1] for p in train_p])
    y_train = np.array([targets[p][0] for p in train_p])
    X_val = np.array([targets[p][1] for p in val_p])
    y_val = np.array([targets[p][0] for p in val_p])
    
    xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, 
                                 early_stopping_rounds=10, random_state=config.SEED)
    xgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    y_pred = xgb_model.predict(X_val)
    
    baseline_r2 = r2_score(y_val, y_pred)
    baseline_mae = mean_absolute_error(y_val, y_pred)
    print(f"Baseline XGBoost R²: {baseline_r2:.4f}, MAE: {baseline_mae:.4f}")
    
    # --- 2. Deep Learning Model ---
    print("\n--- Training Full Multimodal Model (v1.txt replication) ---")
    
    train_ds = OptimizedOSICDataset(train_p, targets, config.TRAIN_DIR, 'train')
    val_ds = OptimizedOSICDataset(val_p, targets, config.TRAIN_DIR, 'val')
    
    train_loader = DataLoader(train_ds, batch_size=config.BATCH_SIZE, shuffle=True, num_workers=config.NUM_WORKERS, pin_memory=True, drop_last=True)
    val_loader = DataLoader(val_ds, batch_size=config.BATCH_SIZE, shuffle=False, num_workers=config.NUM_WORKERS)
    
    model = OptimizedDenseNetModel(tabular_dim=tabular_dim).to(config.DEVICE)
    criterion = RegressionLoss()
    
    optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), 
                                  lr=config.LR, weight_decay=config.WEIGHT_DECAY)
    
    # --- Use ReduceLROnPlateau as in v1.txt ---
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, 
                                  patience=config.SCHEDULER_PATIENCE, verbose=True)
    scaler = GradScaler()
    
    best_r2 = -float('inf')
    best_mae = float('inf')
    best_lll = -float('inf')
    patience_counter = 0
    
    for epoch in range(config.EPOCHS):
        print(f"\nEpoch {epoch+1}/{config.EPOCHS}")
        
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion, scaler)
        print(f"Train Loss: {train_loss:.4f}")
        
        val_metrics = validate_one_epoch(model, val_loader)
        
        print("--- Validation Metrics ---")
        print(f"  R²: {val_metrics['R2']:.4f}  |  MAE: {val_metrics['MAE']:.4f}  |  RMSE: {val_metrics['RMSE']:.4f}")
        print(f"  LLL: {val_metrics['LaplaceLL']:.4f}")
        print("--------------------------")
        
        scheduler.step(val_metrics['R2']) 
        
        if val_metrics['R2'] > best_r2:
            best_r2 = val_metrics['R2']
            best_mae = val_metrics['MAE']
            best_lll = val_metrics['LaplaceLL']
            print(f"🎯 New Best Model! R²: {best_r2:.4f}, MAE: {best_mae:.4f}, LLL: {best_lll:.4f}. Saving...")
            torch.save(model.state_dict(), config.OUTPUT_DIR / 'best_v1_replication_model.pth')
            patience_counter = 0
        else:
            patience_counter += 1
        
        if patience_counter >= config.EARLY_STOP_PATIENCE:
            print(f"Early stopping at epoch {epoch+1}")
            break
            
    print(f"\n{'='*25} FINAL RESULTS (v1.txt replication) {'='*25}")
    print(f"🔥 Best R²: {best_r2:.4f}")
    print(f"🔥 Best MAE: {best_mae:.4f}")
    print(f"🔥 Best LLL: {best_lll:.4f}")
    
    print("\n--- Baseline Comparison ---")
    print(f"Baseline R²: {baseline_r2:.4f}")
    print(f"Model R²:    {best_r2:.4f}")
    print(f"Improvement: {best_r2 - baseline_r2:.4f}")

if __name__ == "__main__":
    run()

Loading data and calculating targets...


100%|██████████| 176/176 [00:00<00:00, 1291.24it/s]

Processed 176 patients.
Detected Tabular Dimension: 9

--- Single Split (v1.txt replication) ---
Train: 149 patients, Val: 27 patients

--- Training Clinical Baseline (XGBoost) ---


Baseline XGBoost R²: 0.0325, MAE: 4.9204

--- Training Full Multimodal Model (v1.txt replication) ---
Dataset split 'train': 149 valid patients with images.


Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


...

--- Validation Metrics ---
  R²: 0.1101  |  MAE: 4.4249  |  RMSE: 5.1337
  LLL: -4.6858
--------------------------

Epoch 21/50


Train Loss: 3.8742


--- Validation Metrics ---
  R²: -0.0970  |  MAE: 4.6161  |  RMSE: 5.7000
  LLL: -4.8311
--------------------------

Epoch 22/50


Train Loss: 3.9107


--- Validation Metrics ---
  R²: -0.1374  |  MAE: 4.9907  |  RMSE: 5.8040
  LLL: -5.1624
--------------------------
Early stopping at epoch 22

========================= FINAL RESULTS (v1.txt replication) =========================
🔥 Best R²: 0.2919
🔥 Best MAE: 4.0861
🔥 Best LLL: -4.4163

--- Baseline Comparison ---
Baseline R²: 0.0325
Model R²:    0.2919
Improvement: 0.2594


In [ ]:
# ============================================================================
# 7. MAIN EXECUTION (Single Split)
# ============================================================================

def run():
    df, targets, baseline_df, tabular_dim = prepare_data()
    
    patients = baseline_df['Patient'].values
    bins = baseline_df['decay_bins'].values
    
    # --- Create the v1.txt split ---
    train_p, val_p = train_test_split(
        patients, 
        test_size=config.TEST_SIZE, 
        random_state=config.SEED, 
        stratify=bins
    )
    
    print(f"\n--- Single Split (v1.txt replication) ---")
    print(f"Train: {len(train_p)} patients, Val: {len(val_p)} patients")
    
    # --- 1. Clinical Baseline (XGBoost) ---
    print("\n--- Training Clinical Baseline (XGBoost) ---")
    
    # --- *** FIX HERE *** ---
    # `targets[p][1]` is tabular_features
    # `targets[p][0]` is slope
    X_train = np.array([targets[p][1] for p in train_p])
    y_train = np.array([targets[p][0] for p in train_p])
    X_val = np.array([targets[p][1] for p in val_p])
    y_val = np.array([targets[p][0] for p in val_p])
    
    xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, 
                                 early_stopping_rounds=10, random_state=config.SEED)
    xgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    y_pred = xgb_model.predict(X_val)
    
    baseline_r2 = r2_score(y_val, y_pred)
    baseline_mae = mean_absolute_error(y_val, y_pred)
    print(f"Baseline XGBoost R²: {baseline_r2:.4f}, MAE: {baseline_mae:.4f}")
    
    # --- 2. Deep Learning Model ---
    print("\n--- Training Full Multimodal Model (v1.txt replication) ---")
    
    train_ds = OptimizedOSICDataset(train_p, targets, config.TRAIN_DIR, 'train')
    val_ds = OptimizedOSICDataset(val_p, targets, config.TRAIN_DIR, 'val')
    
    train_loader = DataLoader(train_ds, batch_size=config.BATCH_SIZE, shuffle=True, num_workers=config.NUM_WORKERS, pin_memory=True, drop_last=True)
    val_loader = DataLoader(val_ds, batch_size=config.BATCH_SIZE, shuffle=False, num_workers=config.NUM_WORKERS)
    
    model = OptimizedDenseNetModel(tabular_dim=tabular_dim).to(config.DEVICE)
    criterion = RegressionLoss()
    
    optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), 
                                  lr=config.LR, weight_decay=config.WEIGHT_DECAY)
    
    # --- Use ReduceLROnPlateau as in v1.txt ---
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, 
                                  patience=config.SCHEDULER_PATIENCE, verbose=True)
    scaler = GradScaler()
    
    best_r2 = -float('inf')
    best_mae = float('inf')
    best_lll = -float('inf')
    patience_counter = 0
    
    for epoch in range(config.EPOCHS):
        print(f"\nEpoch {epoch+1}/{config.EPOCHS}")
        
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion, scaler)
        print(f"Train Loss: {train_loss:.4f}")
        
        val_metrics = validate_one_epoch(model, val_loader)
        
        print("--- Validation Metrics ---")
        print(f"  R²: {val_metrics['R2']:.4f}  |  MAE: {val_metrics['MAE']:.4f}  |  RMSE: {val_metrics['RMSE']:.4f}")
        print(f"  LLL: {val_metrics['LaplaceLL']:.4f}")
        print("--------------------------")
        
        scheduler.step(val_metrics['R2']) 
        
        if val_metrics['R2'] > best_r2:
            best_r2 = val_metrics['R2']
            best_mae = val_metrics['MAE']
            best_lll = val_metrics['LaplaceLL']
            print(f"🎯 New Best Model! R²: {best_r2:.4f}, MAE: {best_mae:.4f}, LLL: {best_lll:.4f}. Saving...")
            torch.save(model.state_dict(), config.OUTPUT_DIR / 'best_v2_replication_model.pth')
            patience_counter = 0
        else:
            patience_counter += 1
        
        if patience_counter >= config.EARLY_STOP_PATIENCE:
            print(f"Early stopping at epoch {epoch+1}")
            break
            
    print(f"\n{'='*25} FINAL RESULTS (v1.txt replication) {'='*25}")
    print(f"🔥 Best R²: {best_r2:.4f}")
    print(f"🔥 Best MAE: {best_mae:.4f}")
    print(f"🔥 Best LLL: {best_lll:.4f}")
    
    print("\n--- Baseline Comparison ---")
    print(f"Baseline R²: {baseline_r2:.4f}")
    print(f"Model R²:    {best_r2:.4f}")
    print(f"Improvement: {best_r2 - baseline_r2:.4f}")

if __name__ == "__main__":
    run()

Loading data and calculating targets...


100%|██████████| 176/176 [00:00<00:00, 1294.86it/s]


Processed 176 patients.
Detected Tabular Dimension: 9

--- Single Split (v1.txt replication) ---
Train: 149 patients, Val: 27 patients

--- Training Clinical Baseline (XGBoost) ---
Baseline XGBoost R²: 0.0325, MAE: 4.9204

--- Training Full Multimodal Model (v1.txt replication) ---
Dataset split 'train': 149 valid patients with images.
Dataset split 'val': 25 valid patients with images.
Applying partial freezing (like v1.txt): only first 100 layers frozen.

Epoch 1/50


Train Loss: 28.6541


--- Validation Metrics ---
  R²: -0.0791  |  MAE: 4.7034  |  RMSE: 5.6533
  LLL: -6.1278
--------------------------
🎯 New Best Model! R²: -0.0791, MAE: 4.7034, LLL: -6.1278. Saving...

Epoch 2/50


...

Train Loss: 4.5943


--- Validation Metrics ---
  R²: 0.0641  |  MAE: 4.2278  |  RMSE: 5.2648
  LLL: -4.5566
--------------------------

Epoch 16/50


Train Loss: 4.5109


--- Validation Metrics ---
  R²: -0.1047  |  MAE: 4.9025  |  RMSE: 5.7199
  LLL: -5.0678
--------------------------

Epoch 17/50


Training:  81%|████████  | 121/149 [00:12<00:02,  9.77it/s, loss=3.33] 

In [ ]:
# ============================================================================
# 7. MAIN EXECUTION (Single Split)
# ============================================================================

def run():
    df, targets, baseline_df, tabular_dim = prepare_data()
    
    patients = baseline_df['Patient'].values
    bins = baseline_df['decay_bins'].values
    
    # --- Create the v1.txt split ---
    train_p, val_p = train_test_split(
        patients, 
        test_size=config.TEST_SIZE, 
        random_state=config.SEED, 
        stratify=bins
    )
    
    print(f"\n--- Single Split (v1.txt replication) ---")
    print(f"Train: {len(train_p)} patients, Val: {len(val_p)} patients")
    
    # --- 1. Clinical Baseline (XGBoost) ---
    print("\n--- Training Clinical Baseline (XGBoost) ---")
    
    # --- *** FIX HERE *** ---
    # `targets[p][1]` is tabular_features
    # `targets[p][0]` is slope
    X_train = np.array([targets[p][1] for p in train_p])
    y_train = np.array([targets[p][0] for p in train_p])
    X_val = np.array([targets[p][1] for p in val_p])
    y_val = np.array([targets[p][0] for p in val_p])
    
    xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, 
                                 early_stopping_rounds=10, random_state=config.SEED)
    xgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    y_pred = xgb_model.predict(X_val)
    
    baseline_r2 = r2_score(y_val, y_pred)
    baseline_mae = mean_absolute_error(y_val, y_pred)
    print(f"Baseline XGBoost R²: {baseline_r2:.4f}, MAE: {baseline_mae:.4f}")
    
    # --- 2. Deep Learning Model ---
    print("\n--- Training Full Multimodal Model (v1.txt replication) ---")
    
    train_ds = OptimizedOSICDataset(train_p, targets, config.TRAIN_DIR, 'train')
    val_ds = OptimizedOSICDataset(val_p, targets, config.TRAIN_DIR, 'val')
    
    train_loader = DataLoader(train_ds, batch_size=config.BATCH_SIZE, shuffle=True, num_workers=config.NUM_WORKERS, pin_memory=True, drop_last=True)
    val_loader = DataLoader(val_ds, batch_size=config.BATCH_SIZE, shuffle=False, num_workers=config.NUM_WORKERS)
    
    model = OptimizedDenseNetModel(tabular_dim=tabular_dim).to(config.DEVICE)
    criterion = RegressionLoss()
    
    optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), 
                                  lr=config.LR, weight_decay=config.WEIGHT_DECAY)
    
    # --- Use ReduceLROnPlateau as in v1.txt ---
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, 
                                  patience=config.SCHEDULER_PATIENCE, verbose=True)
    scaler = GradScaler()
    
    best_r2 = -float('inf')
    best_mae = float('inf')
    best_lll = -float('inf')
    patience_counter = 0
    
    for epoch in range(config.EPOCHS):
        print(f"\nEpoch {epoch+1}/{config.EPOCHS}")
        
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion, scaler)
        print(f"Train Loss: {train_loss:.4f}")
        
        val_metrics = validate_one_epoch(model, val_loader)
        
        print("--- Validation Metrics ---")
        print(f"  R²: {val_metrics['R2']:.4f}  |  MAE: {val_metrics['MAE']:.4f}  |  RMSE: {val_metrics['RMSE']:.4f}")
        print(f"  LLL: {val_metrics['LaplaceLL']:.4f}")
        print("--------------------------")
        
        scheduler.step(val_metrics['R2']) 
        
        if val_metrics['R2'] > best_r2:
            best_r2 = val_metrics['R2']
            best_mae = val_metrics['MAE']
            best_lll = val_metrics['LaplaceLL']
            print(f"🎯 New Best Model! R²: {best_r2:.4f}, MAE: {best_mae:.4f}, LLL: {best_lll:.4f}. Saving...")
            torch.save(model.state_dict(), config.OUTPUT_DIR / 'best_v3_replication_model.pth')
            patience_counter = 0
        else:
            patience_counter += 1
        
        if patience_counter >= config.EARLY_STOP_PATIENCE:
            print(f"Early stopping at epoch {epoch+1}")
            break
            
    print(f"\n{'='*25} FINAL RESULTS (v1.txt replication) {'='*25}")
    print(f"🔥 Best R²: {best_r2:.4f}")
    print(f"🔥 Best MAE: {best_mae:.4f}")
    print(f"🔥 Best LLL: {best_lll:.4f}")
    
    print("\n--- Baseline Comparison ---")
    print(f"Baseline R²: {baseline_r2:.4f}")
    print(f"Model R²:    {best_r2:.4f}")
    print(f"Improvement: {best_r2 - baseline_r2:.4f}")

if __name__ == "__main__":
    run()

In [ ]:
# ============================================================================
# 7. MAIN EXECUTION (Single Split)
# ============================================================================

def run():
    df, targets, baseline_df, tabular_dim = prepare_data()
    
    patients = baseline_df['Patient'].values
    bins = baseline_df['decay_bins'].values
    
    # --- Create the v1.txt split ---
    train_p, val_p = train_test_split(
        patients, 
        test_size=config.TEST_SIZE, 
        random_state=config.SEED, 
        stratify=bins
    )
    
    print(f"\n--- Single Split (v1.txt replication) ---")
    print(f"Train: {len(train_p)} patients, Val: {len(val_p)} patients")
    
    # --- 1. Clinical Baseline (XGBoost) ---
    print("\n--- Training Clinical Baseline (XGBoost) ---")
    
    # --- *** FIX HERE *** ---
    # `targets[p][1]` is tabular_features
    # `targets[p][0]` is slope
    X_train = np.array([targets[p][1] for p in train_p])
    y_train = np.array([targets[p][0] for p in train_p])
    X_val = np.array([targets[p][1] for p in val_p])
    y_val = np.array([targets[p][0] for p in val_p])
    
    xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, 
                                 early_stopping_rounds=10, random_state=config.SEED)
    xgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    y_pred = xgb_model.predict(X_val)
    
    baseline_r2 = r2_score(y_val, y_pred)
    baseline_mae = mean_absolute_error(y_val, y_pred)
    print(f"Baseline XGBoost R²: {baseline_r2:.4f}, MAE: {baseline_mae:.4f}")
    
    # --- 2. Deep Learning Model ---
    print("\n--- Training Full Multimodal Model (v1.txt replication) ---")
    
    train_ds = OptimizedOSICDataset(train_p, targets, config.TRAIN_DIR, 'train')
    val_ds = OptimizedOSICDataset(val_p, targets, config.TRAIN_DIR, 'val')
    
    train_loader = DataLoader(train_ds, batch_size=config.BATCH_SIZE, shuffle=True, num_workers=config.NUM_WORKERS, pin_memory=True, drop_last=True)
    val_loader = DataLoader(val_ds, batch_size=config.BATCH_SIZE, shuffle=False, num_workers=config.NUM_WORKERS)
    
    model = OptimizedDenseNetModel(tabular_dim=tabular_dim).to(config.DEVICE)
    criterion = RegressionLoss()
    
    optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), 
                                  lr=config.LR, weight_decay=config.WEIGHT_DECAY)
    
    # --- Use ReduceLROnPlateau as in v1.txt ---
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, 
                                  patience=config.SCHEDULER_PATIENCE, verbose=True)
    scaler = GradScaler()
    
    best_r2 = -float('inf')
    best_mae = float('inf')
    best_lll = -float('inf')
    patience_counter = 0
    
    for epoch in range(config.EPOCHS):
        print(f"\nEpoch {epoch+1}/{config.EPOCHS}")
        
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion, scaler)
        print(f"Train Loss: {train_loss:.4f}")
        
        val_metrics = validate_one_epoch(model, val_loader)
        
        print("--- Validation Metrics ---")
        print(f"  R²: {val_metrics['R2']:.4f}  |  MAE: {val_metrics['MAE']:.4f}  |  RMSE: {val_metrics['RMSE']:.4f}")
        print(f"  LLL: {val_metrics['LaplaceLL']:.4f}")
        print("--------------------------")
        
        scheduler.step(val_metrics['R2']) 
        
        if val_metrics['R2'] > best_r2:
            best_r2 = val_metrics['R2']
            best_mae = val_metrics['MAE']
            best_lll = val_metrics['LaplaceLL']
            print(f"🎯 New Best Model! R²: {best_r2:.4f}, MAE: {best_mae:.4f}, LLL: {best_lll:.4f}. Saving...")
            torch.save(model.state_dict(), config.OUTPUT_DIR / 'best_v4_replication_model.pth')
            patience_counter = 0
        else:
            patience_counter += 1
        
        if patience_counter >= config.EARLY_STOP_PATIENCE:
            print(f"Early stopping at epoch {epoch+1}")
            break
            
    print(f"\n{'='*25} FINAL RESULTS (v1.txt replication) {'='*25}")
    print(f"🔥 Best R²: {best_r2:.4f}")
    print(f"🔥 Best MAE: {best_mae:.4f}")
    print(f"🔥 Best LLL: {best_lll:.4f}")
    
    print("\n--- Baseline Comparison ---")
    print(f"Baseline R²: {baseline_r2:.4f}")
    print(f"Model R²:    {best_r2:.4f}")
    print(f"Improvement: {best_r2 - baseline_r2:.4f}")

if __name__ == "__main__":
    run()

In [ ]:
# ============================================================================
# 7. MAIN EXECUTION (Single Split)
# ============================================================================

def run():
    df, targets, baseline_df, tabular_dim = prepare_data()
    
    patients = baseline_df['Patient'].values
    bins = baseline_df['decay_bins'].values
    
    # --- Create the v1.txt split ---
    train_p, val_p = train_test_split(
        patients, 
        test_size=config.TEST_SIZE, 
        random_state=config.SEED, 
        stratify=bins
    )
    
    print(f"\n--- Single Split (v1.txt replication) ---")
    print(f"Train: {len(train_p)} patients, Val: {len(val_p)} patients")
    
    # --- 1. Clinical Baseline (XGBoost) ---
    print("\n--- Training Clinical Baseline (XGBoost) ---")
    
    # --- *** FIX HERE *** ---
    # `targets[p][1]` is tabular_features
    # `targets[p][0]` is slope
    X_train = np.array([targets[p][1] for p in train_p])
    y_train = np.array([targets[p][0] for p in train_p])
    X_val = np.array([targets[p][1] for p in val_p])
    y_val = np.array([targets[p][0] for p in val_p])
    
    xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, 
                                 early_stopping_rounds=10, random_state=config.SEED)
    xgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    y_pred = xgb_model.predict(X_val)
    
    baseline_r2 = r2_score(y_val, y_pred)
    baseline_mae = mean_absolute_error(y_val, y_pred)
    print(f"Baseline XGBoost R²: {baseline_r2:.4f}, MAE: {baseline_mae:.4f}")
    
    # --- 2. Deep Learning Model ---
    print("\n--- Training Full Multimodal Model (v1.txt replication) ---")
    
    train_ds = OptimizedOSICDataset(train_p, targets, config.TRAIN_DIR, 'train')
    val_ds = OptimizedOSICDataset(val_p, targets, config.TRAIN_DIR, 'val')
    
    train_loader = DataLoader(train_ds, batch_size=config.BATCH_SIZE, shuffle=True, num_workers=config.NUM_WORKERS, pin_memory=True, drop_last=True)
    val_loader = DataLoader(val_ds, batch_size=config.BATCH_SIZE, shuffle=False, num_workers=config.NUM_WORKERS)
    
    model = OptimizedDenseNetModel(tabular_dim=tabular_dim).to(config.DEVICE)
    criterion = RegressionLoss()
    
    optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), 
                                  lr=config.LR, weight_decay=config.WEIGHT_DECAY)
    
    # --- Use ReduceLROnPlateau as in v1.txt ---
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, 
                                  patience=config.SCHEDULER_PATIENCE, verbose=True)
    scaler = GradScaler()
    
    best_r2 = -float('inf')
    best_mae = float('inf')
    best_lll = -float('inf')
    patience_counter = 0
    
    for epoch in range(config.EPOCHS):
        print(f"\nEpoch {epoch+1}/{config.EPOCHS}")
        
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion, scaler)
        print(f"Train Loss: {train_loss:.4f}")
        
        val_metrics = validate_one_epoch(model, val_loader)
        
        print("--- Validation Metrics ---")
        print(f"  R²: {val_metrics['R2']:.4f}  |  MAE: {val_metrics['MAE']:.4f}  |  RMSE: {val_metrics['RMSE']:.4f}")
        print(f"  LLL: {val_metrics['LaplaceLL']:.4f}")
        print("--------------------------")
        
        scheduler.step(val_metrics['R2']) 
        
        if val_metrics['R2'] > best_r2:
            best_r2 = val_metrics['R2']
            best_mae = val_metrics['MAE']
            best_lll = val_metrics['LaplaceLL']
            print(f"🎯 New Best Model! R²: {best_r2:.4f}, MAE: {best_mae:.4f}, LLL: {best_lll:.4f}. Saving...")
            torch.save(model.state_dict(), config.OUTPUT_DIR / 'best_v5_replication_model.pth')
            patience_counter = 0
        else:
            patience_counter += 1
        
        if patience_counter >= config.EARLY_STOP_PATIENCE:
            print(f"Early stopping at epoch {epoch+1}")
            break
            
    print(f"\n{'='*25} FINAL RESULTS (v1.txt replication) {'='*25}")
    print(f"🔥 Best R²: {best_r2:.4f}")
    print(f"🔥 Best MAE: {best_mae:.4f}")
    print(f"🔥 Best LLL: {best_lll:.4f}")
    
    print("\n--- Baseline Comparison ---")
    print(f"Baseline R²: {baseline_r2:.4f}")
    print(f"Model R²:    {best_r2:.4f}")
    print(f"Improvement: {best_r2 - baseline_r2:.4f}")

if __name__ == "__main__":
    run()

In [ ]:
# ============================================================================
# 7. MAIN EXECUTION (Single Split)
# ============================================================================

def run():
    df, targets, baseline_df, tabular_dim = prepare_data()
    
    patients = baseline_df['Patient'].values
    bins = baseline_df['decay_bins'].values
    
    # --- Create the v1.txt split ---
    train_p, val_p = train_test_split(
        patients, 
        test_size=config.TEST_SIZE, 
        random_state=config.SEED, 
        stratify=bins
    )
    
    print(f"\n--- Single Split (v1.txt replication) ---")
    print(f"Train: {len(train_p)} patients, Val: {len(val_p)} patients")
    
    # --- 1. Clinical Baseline (XGBoost) ---
    print("\n--- Training Clinical Baseline (XGBoost) ---")
    
    # --- *** FIX HERE *** ---
    # `targets[p][1]` is tabular_features
    # `targets[p][0]` is slope
    X_train = np.array([targets[p][1] for p in train_p])
    y_train = np.array([targets[p][0] for p in train_p])
    X_val = np.array([targets[p][1] for p in val_p])
    y_val = np.array([targets[p][0] for p in val_p])
    
    xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, 
                                 early_stopping_rounds=10, random_state=config.SEED)
    xgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    y_pred = xgb_model.predict(X_val)
    
    baseline_r2 = r2_score(y_val, y_pred)
    baseline_mae = mean_absolute_error(y_val, y_pred)
    print(f"Baseline XGBoost R²: {baseline_r2:.4f}, MAE: {baseline_mae:.4f}")
    
    # --- 2. Deep Learning Model ---
    print("\n--- Training Full Multimodal Model (v1.txt replication) ---")
    
    train_ds = OptimizedOSICDataset(train_p, targets, config.TRAIN_DIR, 'train')
    val_ds = OptimizedOSICDataset(val_p, targets, config.TRAIN_DIR, 'val')
    
    train_loader = DataLoader(train_ds, batch_size=config.BATCH_SIZE, shuffle=True, num_workers=config.NUM_WORKERS, pin_memory=True, drop_last=True)
    val_loader = DataLoader(val_ds, batch_size=config.BATCH_SIZE, shuffle=False, num_workers=config.NUM_WORKERS)
    
    model = OptimizedDenseNetModel(tabular_dim=tabular_dim).to(config.DEVICE)
    criterion = RegressionLoss()
    
    optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), 
                                  lr=config.LR, weight_decay=config.WEIGHT_DECAY)
    
    # --- Use ReduceLROnPlateau as in v1.txt ---
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, 
                                  patience=config.SCHEDULER_PATIENCE, verbose=True)
    scaler = GradScaler()
    
    best_r2 = -float('inf')
    best_mae = float('inf')
    best_lll = -float('inf')
    patience_counter = 0
    
    for epoch in range(config.EPOCHS):
        print(f"\nEpoch {epoch+1}/{config.EPOCHS}")
        
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion, scaler)
        print(f"Train Loss: {train_loss:.4f}")
        
        val_metrics = validate_one_epoch(model, val_loader)
        
        print("--- Validation Metrics ---")
        print(f"  R²: {val_metrics['R2']:.4f}  |  MAE: {val_metrics['MAE']:.4f}  |  RMSE: {val_metrics['RMSE']:.4f}")
        print(f"  LLL: {val_metrics['LaplaceLL']:.4f}")
        print("--------------------------")
        
        scheduler.step(val_metrics['R2']) 
        
        if val_metrics['R2'] > best_r2:
            best_r2 = val_metrics['R2']
            best_mae = val_metrics['MAE']
            best_lll = val_metrics['LaplaceLL']
            print(f"🎯 New Best Model! R²: {best_r2:.4f}, MAE: {best_mae:.4f}, LLL: {best_lll:.4f}. Saving...")
            torch.save(model.state_dict(), config.OUTPUT_DIR / 'best_v6_replication_model.pth')
            patience_counter = 0
        else:
            patience_counter += 1
        
        if patience_counter >= config.EARLY_STOP_PATIENCE:
            print(f"Early stopping at epoch {epoch+1}")
            break
            
    print(f"\n{'='*25} FINAL RESULTS (v1.txt replication) {'='*25}")
    print(f"🔥 Best R²: {best_r2:.4f}")
    print(f"🔥 Best MAE: {best_mae:.4f}")
    print(f"🔥 Best LLL: {best_lll:.4f}")
    
    print("\n--- Baseline Comparison ---")
    print(f"Baseline R²: {baseline_r2:.4f}")
    print(f"Model R²:    {best_r2:.4f}")
    print(f"Improvement: {best_r2 - baseline_r2:.4f}")

if __name__ == "__main__":
    run()

In [ ]:
# ============================================================================
# 7. MAIN EXECUTION (Single Split)
# ============================================================================

def run():
    df, targets, baseline_df, tabular_dim = prepare_data()
    
    patients = baseline_df['Patient'].values
    bins = baseline_df['decay_bins'].values
    
    # --- Create the v1.txt split ---
    train_p, val_p = train_test_split(
        patients, 
        test_size=config.TEST_SIZE, 
        random_state=config.SEED, 
        stratify=bins
    )
    
    print(f"\n--- Single Split (v1.txt replication) ---")
    print(f"Train: {len(train_p)} patients, Val: {len(val_p)} patients")
    
    # --- 1. Clinical Baseline (XGBoost) ---
    print("\n--- Training Clinical Baseline (XGBoost) ---")
    
    # --- *** FIX HERE *** ---
    # `targets[p][1]` is tabular_features
    # `targets[p][0]` is slope
    X_train = np.array([targets[p][1] for p in train_p])
    y_train = np.array([targets[p][0] for p in train_p])
    X_val = np.array([targets[p][1] for p in val_p])
    y_val = np.array([targets[p][0] for p in val_p])
    
    xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, 
                                 early_stopping_rounds=10, random_state=config.SEED)
    xgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    y_pred = xgb_model.predict(X_val)
    
    baseline_r2 = r2_score(y_val, y_pred)
    baseline_mae = mean_absolute_error(y_val, y_pred)
    print(f"Baseline XGBoost R²: {baseline_r2:.4f}, MAE: {baseline_mae:.4f}")
    
    # --- 2. Deep Learning Model ---
    print("\n--- Training Full Multimodal Model (v1.txt replication) ---")
    
    train_ds = OptimizedOSICDataset(train_p, targets, config.TRAIN_DIR, 'train')
    val_ds = OptimizedOSICDataset(val_p, targets, config.TRAIN_DIR, 'val')
    
    train_loader = DataLoader(train_ds, batch_size=config.BATCH_SIZE, shuffle=True, num_workers=config.NUM_WORKERS, pin_memory=True, drop_last=True)
    val_loader = DataLoader(val_ds, batch_size=config.BATCH_SIZE, shuffle=False, num_workers=config.NUM_WORKERS)
    
    model = OptimizedDenseNetModel(tabular_dim=tabular_dim).to(config.DEVICE)
    criterion = RegressionLoss()
    
    optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), 
                                  lr=config.LR, weight_decay=config.WEIGHT_DECAY)
    
    # --- Use ReduceLROnPlateau as in v1.txt ---
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, 
                                  patience=config.SCHEDULER_PATIENCE, verbose=True)
    scaler = GradScaler()
    
    best_r2 = -float('inf')
    best_mae = float('inf')
    best_lll = -float('inf')
    patience_counter = 0
    
    for epoch in range(config.EPOCHS):
        print(f"\nEpoch {epoch+1}/{config.EPOCHS}")
        
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion, scaler)
        print(f"Train Loss: {train_loss:.4f}")
        
        val_metrics = validate_one_epoch(model, val_loader)
        
        print("--- Validation Metrics ---")
        print(f"  R²: {val_metrics['R2']:.4f}  |  MAE: {val_metrics['MAE']:.4f}  |  RMSE: {val_metrics['RMSE']:.4f}")
        print(f"  LLL: {val_metrics['LaplaceLL']:.4f}")
        print("--------------------------")
        
        scheduler.step(val_metrics['R2']) 
        
        if val_metrics['R2'] > best_r2:
            best_r2 = val_metrics['R2']
            best_mae = val_metrics['MAE']
            best_lll = val_metrics['LaplaceLL']
            print(f"🎯 New Best Model! R²: {best_r2:.4f}, MAE: {best_mae:.4f}, LLL: {best_lll:.4f}. Saving...")
            torch.save(model.state_dict(), config.OUTPUT_DIR / 'best_v7_replication_model.pth')
            patience_counter = 0
        else:
            patience_counter += 1
        
        if patience_counter >= config.EARLY_STOP_PATIENCE:
            print(f"Early stopping at epoch {epoch+1}")
            break
            
    print(f"\n{'='*25} FINAL RESULTS (v1.txt replication) {'='*25}")
    print(f"🔥 Best R²: {best_r2:.4f}")
    print(f"🔥 Best MAE: {best_mae:.4f}")
    print(f"🔥 Best LLL: {best_lll:.4f}")
    
    print("\n--- Baseline Comparison ---")
    print(f"Baseline R²: {baseline_r2:.4f}")
    print(f"Model R²:    {best_r2:.4f}")
    print(f"Improvement: {best_r2 - baseline_r2:.4f}")

if __name__ == "__main__":
    run()

In [ ]:
# ============================================================================
# 7. MAIN EXECUTION (Single Split)
# ============================================================================

def run():
    df, targets, baseline_df, tabular_dim = prepare_data()
    
    patients = baseline_df['Patient'].values
    bins = baseline_df['decay_bins'].values
    
    # --- Create the v1.txt split ---
    train_p, val_p = train_test_split(
        patients, 
        test_size=config.TEST_SIZE, 
        random_state=config.SEED, 
        stratify=bins
    )
    
    print(f"\n--- Single Split (v1.txt replication) ---")
    print(f"Train: {len(train_p)} patients, Val: {len(val_p)} patients")
    
    # --- 1. Clinical Baseline (XGBoost) ---
    print("\n--- Training Clinical Baseline (XGBoost) ---")
    
    # --- *** FIX HERE *** ---
    # `targets[p][1]` is tabular_features
    # `targets[p][0]` is slope
    X_train = np.array([targets[p][1] for p in train_p])
    y_train = np.array([targets[p][0] for p in train_p])
    X_val = np.array([targets[p][1] for p in val_p])
    y_val = np.array([targets[p][0] for p in val_p])
    
    xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, 
                                 early_stopping_rounds=10, random_state=config.SEED)
    xgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    y_pred = xgb_model.predict(X_val)
    
    baseline_r2 = r2_score(y_val, y_pred)
    baseline_mae = mean_absolute_error(y_val, y_pred)
    print(f"Baseline XGBoost R²: {baseline_r2:.4f}, MAE: {baseline_mae:.4f}")
    
    # --- 2. Deep Learning Model ---
    print("\n--- Training Full Multimodal Model (v1.txt replication) ---")
    
    train_ds = OptimizedOSICDataset(train_p, targets, config.TRAIN_DIR, 'train')
    val_ds = OptimizedOSICDataset(val_p, targets, config.TRAIN_DIR, 'val')
    
    train_loader = DataLoader(train_ds, batch_size=config.BATCH_SIZE, shuffle=True, num_workers=config.NUM_WORKERS, pin_memory=True, drop_last=True)
    val_loader = DataLoader(val_ds, batch_size=config.BATCH_SIZE, shuffle=False, num_workers=config.NUM_WORKERS)
    
    model = OptimizedDenseNetModel(tabular_dim=tabular_dim).to(config.DEVICE)
    criterion = RegressionLoss()
    
    optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), 
                                  lr=config.LR, weight_decay=config.WEIGHT_DECAY)
    
    # --- Use ReduceLROnPlateau as in v1.txt ---
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, 
                                  patience=config.SCHEDULER_PATIENCE, verbose=True)
    scaler = GradScaler()
    
    best_r2 = -float('inf')
    best_mae = float('inf')
    best_lll = -float('inf')
    patience_counter = 0
    
    for epoch in range(config.EPOCHS):
        print(f"\nEpoch {epoch+1}/{config.EPOCHS}")
        
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion, scaler)
        print(f"Train Loss: {train_loss:.4f}")
        
        val_metrics = validate_one_epoch(model, val_loader)
        
        print("--- Validation Metrics ---")
        print(f"  R²: {val_metrics['R2']:.4f}  |  MAE: {val_metrics['MAE']:.4f}  |  RMSE: {val_metrics['RMSE']:.4f}")
        print(f"  LLL: {val_metrics['LaplaceLL']:.4f}")
        print("--------------------------")
        
        scheduler.step(val_metrics['R2']) 
        
        if val_metrics['R2'] > best_r2:
            best_r2 = val_metrics['R2']
            best_mae = val_metrics['MAE']
            best_lll = val_metrics['LaplaceLL']
            print(f"🎯 New Best Model! R²: {best_r2:.4f}, MAE: {best_mae:.4f}, LLL: {best_lll:.4f}. Saving...")
            torch.save(model.state_dict(), config.OUTPUT_DIR / 'best_v8_replication_model.pth')
            patience_counter = 0
        else:
            patience_counter += 1
        
        if patience_counter >= config.EARLY_STOP_PATIENCE:
            print(f"Early stopping at epoch {epoch+1}")
            break
            
    print(f"\n{'='*25} FINAL RESULTS (v1.txt replication) {'='*25}")
    print(f"🔥 Best R²: {best_r2:.4f}")
    print(f"🔥 Best MAE: {best_mae:.4f}")
    print(f"🔥 Best LLL: {best_lll:.4f}")
    
    print("\n--- Baseline Comparison ---")
    print(f"Baseline R²: {baseline_r2:.4f}")
    print(f"Model R²:    {best_r2:.4f}")
    print(f"Improvement: {best_r2 - baseline_r2:.4f}")

if __name__ == "__main__":
    run()

In [ ]:
# ============================================================================
# 7. MAIN EXECUTION (Single Split)
# ============================================================================

def run():
    df, targets, baseline_df, tabular_dim = prepare_data()
    
    patients = baseline_df['Patient'].values
    bins = baseline_df['decay_bins'].values
    
    # --- Create the v1.txt split ---
    train_p, val_p = train_test_split(
        patients, 
        test_size=config.TEST_SIZE, 
        random_state=config.SEED, 
        stratify=bins
    )
    
    print(f"\n--- Single Split (v1.txt replication) ---")
    print(f"Train: {len(train_p)} patients, Val: {len(val_p)} patients")
    
    # --- 1. Clinical Baseline (XGBoost) ---
    print("\n--- Training Clinical Baseline (XGBoost) ---")
    
    # --- *** FIX HERE *** ---
    # `targets[p][1]` is tabular_features
    # `targets[p][0]` is slope
    X_train = np.array([targets[p][1] for p in train_p])
    y_train = np.array([targets[p][0] for p in train_p])
    X_val = np.array([targets[p][1] for p in val_p])
    y_val = np.array([targets[p][0] for p in val_p])
    
    xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, 
                                 early_stopping_rounds=10, random_state=config.SEED)
    xgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    y_pred = xgb_model.predict(X_val)
    
    baseline_r2 = r2_score(y_val, y_pred)
    baseline_mae = mean_absolute_error(y_val, y_pred)
    print(f"Baseline XGBoost R²: {baseline_r2:.4f}, MAE: {baseline_mae:.4f}")
    
    # --- 2. Deep Learning Model ---
    print("\n--- Training Full Multimodal Model (v1.txt replication) ---")
    
    train_ds = OptimizedOSICDataset(train_p, targets, config.TRAIN_DIR, 'train')
    val_ds = OptimizedOSICDataset(val_p, targets, config.TRAIN_DIR, 'val')
    
    train_loader = DataLoader(train_ds, batch_size=config.BATCH_SIZE, shuffle=True, num_workers=config.NUM_WORKERS, pin_memory=True, drop_last=True)
    val_loader = DataLoader(val_ds, batch_size=config.BATCH_SIZE, shuffle=False, num_workers=config.NUM_WORKERS)
    
    model = OptimizedDenseNetModel(tabular_dim=tabular_dim).to(config.DEVICE)
    criterion = RegressionLoss()
    
    optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), 
                                  lr=config.LR, weight_decay=config.WEIGHT_DECAY)
    
    # --- Use ReduceLROnPlateau as in v1.txt ---
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, 
                                  patience=config.SCHEDULER_PATIENCE, verbose=True)
    scaler = GradScaler()
    
    best_r2 = -float('inf')
    best_mae = float('inf')
    best_lll = -float('inf')
    patience_counter = 0
    
    for epoch in range(config.EPOCHS):
        print(f"\nEpoch {epoch+1}/{config.EPOCHS}")
        
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion, scaler)
        print(f"Train Loss: {train_loss:.4f}")
        
        val_metrics = validate_one_epoch(model, val_loader)
        
        print("--- Validation Metrics ---")
        print(f"  R²: {val_metrics['R2']:.4f}  |  MAE: {val_metrics['MAE']:.4f}  |  RMSE: {val_metrics['RMSE']:.4f}")
        print(f"  LLL: {val_metrics['LaplaceLL']:.4f}")
        print("--------------------------")
        
        scheduler.step(val_metrics['R2']) 
        
        if val_metrics['R2'] > best_r2:
            best_r2 = val_metrics['R2']
            best_mae = val_metrics['MAE']
            best_lll = val_metrics['LaplaceLL']
            print(f"🎯 New Best Model! R²: {best_r2:.4f}, MAE: {best_mae:.4f}, LLL: {best_lll:.4f}. Saving...")
            torch.save(model.state_dict(), config.OUTPUT_DIR / 'best_v9_replication_model.pth')
            patience_counter = 0
        else:
            patience_counter += 1
        
        if patience_counter >= config.EARLY_STOP_PATIENCE:
            print(f"Early stopping at epoch {epoch+1}")
            break
            
    print(f"\n{'='*25} FINAL RESULTS (v1.txt replication) {'='*25}")
    print(f"🔥 Best R²: {best_r2:.4f}")
    print(f"🔥 Best MAE: {best_mae:.4f}")
    print(f"🔥 Best LLL: {best_lll:.4f}")
    
    print("\n--- Baseline Comparison ---")
    print(f"Baseline R²: {baseline_r2:.4f}")
    print(f"Model R²:    {best_r2:.4f}")
    print(f"Improvement: {best_r2 - baseline_r2:.4f}")

if __name__ == "__main__":
    run()

In [ ]:
# ============================================================================
# 7. MAIN EXECUTION (Single Split)
# ============================================================================

def run():
    df, targets, baseline_df, tabular_dim = prepare_data()
    
    patients = baseline_df['Patient'].values
    bins = baseline_df['decay_bins'].values
    
    # --- Create the v1.txt split ---
    train_p, val_p = train_test_split(
        patients, 
        test_size=config.TEST_SIZE, 
        random_state=config.SEED, 
        stratify=bins
    )
    
    print(f"\n--- Single Split (v1.txt replication) ---")
    print(f"Train: {len(train_p)} patients, Val: {len(val_p)} patients")
    
    # --- 1. Clinical Baseline (XGBoost) ---
    print("\n--- Training Clinical Baseline (XGBoost) ---")
    
    # --- *** FIX HERE *** ---
    # `targets[p][1]` is tabular_features
    # `targets[p][0]` is slope
    X_train = np.array([targets[p][1] for p in train_p])
    y_train = np.array([targets[p][0] for p in train_p])
    X_val = np.array([targets[p][1] for p in val_p])
    y_val = np.array([targets[p][0] for p in val_p])
    
    xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, 
                                 early_stopping_rounds=10, random_state=config.SEED)
    xgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    y_pred = xgb_model.predict(X_val)
    
    baseline_r2 = r2_score(y_val, y_pred)
    baseline_mae = mean_absolute_error(y_val, y_pred)
    print(f"Baseline XGBoost R²: {baseline_r2:.4f}, MAE: {baseline_mae:.4f}")
    
    # --- 2. Deep Learning Model ---
    print("\n--- Training Full Multimodal Model (v1.txt replication) ---")
    
    train_ds = OptimizedOSICDataset(train_p, targets, config.TRAIN_DIR, 'train')
    val_ds = OptimizedOSICDataset(val_p, targets, config.TRAIN_DIR, 'val')
    
    train_loader = DataLoader(train_ds, batch_size=config.BATCH_SIZE, shuffle=True, num_workers=config.NUM_WORKERS, pin_memory=True, drop_last=True)
    val_loader = DataLoader(val_ds, batch_size=config.BATCH_SIZE, shuffle=False, num_workers=config.NUM_WORKERS)
    
    model = OptimizedDenseNetModel(tabular_dim=tabular_dim).to(config.DEVICE)
    criterion = RegressionLoss()
    
    optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), 
                                  lr=config.LR, weight_decay=config.WEIGHT_DECAY)
    
    # --- Use ReduceLROnPlateau as in v1.txt ---
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, 
                                  patience=config.SCHEDULER_PATIENCE, verbose=True)
    scaler = GradScaler()
    
    best_r2 = -float('inf')
    best_mae = float('inf')
    best_lll = -float('inf')
    patience_counter = 0
    
    for epoch in range(config.EPOCHS):
        print(f"\nEpoch {epoch+1}/{config.EPOCHS}")
        
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion, scaler)
        print(f"Train Loss: {train_loss:.4f}")
        
        val_metrics = validate_one_epoch(model, val_loader)
        
        print("--- Validation Metrics ---")
        print(f"  R²: {val_metrics['R2']:.4f}  |  MAE: {val_metrics['MAE']:.4f}  |  RMSE: {val_metrics['RMSE']:.4f}")
        print(f"  LLL: {val_metrics['LaplaceLL']:.4f}")
        print("--------------------------")
        
        scheduler.step(val_metrics['R2']) 
        
        if val_metrics['R2'] > best_r2:
            best_r2 = val_metrics['R2']
            best_mae = val_metrics['MAE']
            best_lll = val_metrics['LaplaceLL']
            print(f"🎯 New Best Model! R²: {best_r2:.4f}, MAE: {best_mae:.4f}, LLL: {best_lll:.4f}. Saving...")
            torch.save(model.state_dict(), config.OUTPUT_DIR / 'best_v10_replication_model.pth')
            patience_counter = 0
        else:
            patience_counter += 1
        
        if patience_counter >= config.EARLY_STOP_PATIENCE:
            print(f"Early stopping at epoch {epoch+1}")
            break
            
    print(f"\n{'='*25} FINAL RESULTS (v1.txt replication) {'='*25}")
    print(f"🔥 Best R²: {best_r2:.4f}")
    print(f"🔥 Best MAE: {best_mae:.4f}")
    print(f"🔥 Best LLL: {best_lll:.4f}")
    
    print("\n--- Baseline Comparison ---")
    print(f"Baseline R²: {baseline_r2:.4f}")
    print(f"Model R²:    {best_r2:.4f}")
    print(f"Improvement: {best_r2 - baseline_r2:.4f}")

if __name__ == "__main__":
    run()